# Problem 4

In [1]:
import numpy as np
import pandas as pd
from torch import nn 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms.functional as TF
from sklearn import metrics


import os
import cv2
from torch.nn import ConvTranspose2d
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Module
from torch.nn import ModuleList
from torch.nn import ReLU
from torchvision.transforms import CenterCrop
from torch.nn import functional as F
import torch

from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from torchvision import transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import time
import os

In [2]:
BATCH_SIZE = 8

### Data input and train_test_split

In [3]:
data_path_4 = '/Users/charliejiang/Documents/Stanford/Machine-Learning-for-Neuroimaging/HW2/data_assignment_2/lgg-mri-segmentation/kaggle_3m'

metadata = pd.read_csv(data_path_4 + '/data.csv')
patient_id = metadata['Patient']

In [4]:
folder_lst = []
for i in os.listdir(data_path_4):
    if i[0:12] in patient_id.values:
        folder_lst.append(os.path.join(data_path_4,i))
print(len(folder_lst))

110


In [5]:
img_lst = []
mask_lst = []
#Obtain paths for both mask and MRI images at same time, ensuring path order is preservered
for i in folder_lst:
    file_lst = os.listdir(i)
    for j in file_lst:
        if j[-8:] != 'mask.tif':
            img_lst.append(os.path.join(data_path_4,i,j))
            mask_lst.append(os.path.join(data_path_4,i,j[:-4])+'_mask.tif')            

In [6]:
mask_lst[0]

'/Users/charliejiang/Documents/Stanford/Machine-Learning-for-Neuroimaging/HW2/data_assignment_2/lgg-mri-segmentation/kaggle_3m/TCGA_CS_6667_20011105/TCGA_CS_6667_20011105_9_mask.tif'

In [7]:
cv2.imread(img_lst[0])

array([[[1, 1, 0],
        [1, 1, 0],
        [1, 1, 0],
        ...,
        [1, 1, 0],
        [1, 1, 0],
        [1, 1, 0]],

       [[1, 1, 0],
        [1, 1, 0],
        [1, 1, 0],
        ...,
        [1, 1, 0],
        [1, 1, 0],
        [1, 1, 0]],

       [[1, 1, 0],
        [1, 1, 0],
        [1, 1, 0],
        ...,
        [1, 0, 0],
        [1, 0, 0],
        [1, 1, 0]],

       ...,

       [[1, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        ...,
        [1, 1, 0],
        [1, 0, 0],
        [1, 0, 0]],

       [[1, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        ...,
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]],

       [[1, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        ...,
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]]], dtype=uint8)

In [8]:
#Test if input data is standarized
shape_0 =  cv2.imread(img_lst[0]).shape
for i in img_lst:    
    if cv2.imread(i).shape != shape_0:
        print("This aint it")
        break
mshape_0 =  cv2.imread(img_lst[0]).shape    
if mshape_0 != shape_0:
    print("Nah")
for i in mask_lst:    
    if cv2.imread(i).shape != mshape_0:
        print("This aint it")
        break


In [9]:
initial_shape = (shape_0[0],shape_0[1])

In [10]:
train_val_imgs,test_imgs,train_val_masks,test_masks = train_test_split(img_lst,mask_lst,test_size=0.2, random_state=42)
train_imgs,val_imgs,train_masks,val_masks = train_test_split(train_val_imgs,train_val_masks,test_size=0.125, random_state=42)

In [11]:
#Verify split is correct

print(len(train_imgs))
print(len(val_imgs))
print(len(test_imgs))

for i in val_imgs:
    if i in train_imgs or i in test_imgs:
        print('Error')
        break
for j in val_masks:
    if j in train_masks or j in test_masks:
        print('Error')
        break
        
for i in test_imgs:
    if i in train_imgs:
        print('Error')
        break
for j in test_masks:
    if j in train_masks:
        print('Error')
        break
        

2750
393
786


In [12]:
class SegmentationDataset(Dataset):
    def __init__(self, imagePaths, maskPaths):
        self.imagePaths = imagePaths
        self.maskPaths = maskPaths
        self.transforms = transforms
    def __len__(self):
        return len(self.imagePaths)
    def __getitem__(self, idx):
        imagePath = self.imagePaths[idx]
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        mask = cv2.imread(self.maskPaths[idx], 0)
        
        image = TF.to_tensor(image)
        mask = TF.to_tensor(mask)
        
        return (image, mask)

In [13]:
trainDS = SegmentationDataset(imagePaths=train_imgs, maskPaths=train_masks)
valDS = SegmentationDataset(imagePaths=val_imgs, maskPaths=val_masks)
testDS = SegmentationDataset(imagePaths=test_imgs, maskPaths=test_masks)
print(f"[INFO] found {len(trainDS)} examples in the training set...")
print(f"[INFO] found {len(valDS)} examples in the val set...")
print(f"[INFO] found {len(testDS)} examples in the test set...")

trainLoader = DataLoader(trainDS, shuffle=True,batch_size = BATCH_SIZE, pin_memory=True)

valLoader = DataLoader(valDS, shuffle=False,batch_size = BATCH_SIZE,pin_memory=True)

testLoader = DataLoader(testDS, shuffle=False,batch_size = BATCH_SIZE, pin_memory=True)


[INFO] found 2750 examples in the training set...
[INFO] found 393 examples in the val set...
[INFO] found 786 examples in the test set...


In [14]:
class DConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

In [15]:
class Encoder(Module):
    def __init__(self, channels=(3, 16, 32, 64)):
        super().__init__()
        self.encBlocks = ModuleList(
            [DConvBlock(channels[i], channels[i + 1])
                 for i in range(len(channels) - 1)])
        self.pool = MaxPool2d(2)
    def forward(self, x):
        blockOutputs = []
        for block in self.encBlocks:
            x = block(x)
            blockOutputs.append(x)
            x = self.pool(x)
        return blockOutputs

In [16]:
class Decoder(Module):
    def __init__(self, channels=(64, 32, 16)):
        super().__init__()
        self.channels = channels
        self.upconvs = ModuleList(
            [ConvTranspose2d(channels[i], channels[i + 1], 2, 2)
                 for i in range(len(channels) - 1)])
        self.dec_blocks = ModuleList(
            [DConvBlock(channels[i], channels[i + 1])
                for i in range(len(channels) - 1)])
    def forward(self, x, encFeatures):
        for i in range(len(self.channels) - 1):
            x = self.upconvs[i](x)
            encFeat = self.crop(encFeatures[i], x)
            x = torch.cat([x, encFeat], dim=1)
            x = self.dec_blocks[i](x)
        return x
    def crop(self, encFeatures, x):
        (_, _, H, W) = x.shape
        encFeatures = CenterCrop([H, W])(encFeatures)
        return encFeatures


In [17]:
class UNet(Module):
    def __init__(self, encChannels=(3, 16, 32, 64),
                 decChannels=(64, 32, 16),
                 nbClasses=1, retainDim=True,
                 outSize=initial_shape):
        super().__init__()

        self.encoder = Encoder(encChannels)
        self.decoder = Decoder(decChannels)

        self.head = Conv2d(decChannels[-1], nbClasses, 1)
        self.retainDim = retainDim
        self.outSize = outSize
    def forward(self, x):
        encFeatures = self.encoder(x)
        decFeatures = self.decoder(encFeatures[::-1][0],encFeatures[::-1][1:])
        map = self.head(decFeatures)
        if self.retainDim:
            map = F.interpolate(map, self.outSize)
        return map

In [18]:
u_net=UNet()
print(u_net)

UNet(
  (encoder): Encoder(
    (encBlocks): ModuleList(
      (0): DConvBlock(
        (double_conv): Sequential(
          (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
          (3): ReLU(inplace=True)
        )
      )
      (1): DConvBlock(
        (double_conv): Sequential(
          (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
          (3): ReLU(inplace=True)
        )
      )
      (2): DConvBlock(
        (double_conv): Sequential(
          (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
          (3): ReLU(inplace=True)
        )
      )
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (decoder): Decoder(
   

In [19]:
lr = 0.001
lossfun = BCEWithLogitsLoss()
optimizer = Adam(u_net.parameters(), lr)

In [20]:
def jaccard_coeff(y_pred, y_true):
        inter = np.dot(y_pred, y_true)
        union = np.sum(y_pred) + np.sum(y_true) - inter
        jaccard_index = inter/union

        return jaccard_index.cpu().numpy()

In [21]:
def dice_coeff_binary(y_pred, y_true):
        eps = 0.0001
        inter = np.dot(y_pred, y_true)
        union = np.sum(y_pred) + np.sum(y_true) - inter
        return ((2 * inter.float() + eps) / (union.float() + eps)).cpu().numpy()

In [22]:
# loop over epochs
def train_unet(unet,trainLoader,valLoader, opt,lossFunc,epoch_cnt = 100):
    # Training model
    startTime = time.time()
    
    train_loss = []
    valid_loss = []
    train_accuracy = []
    valid_accuracy = []
    train_dice = []
    valid_dice = []
    
    for e in range(epoch_cnt):
        unet.train()
        
        pred_lst = []
        target_lst = []
        pred_val_lst = []
        train_epoch_loss = []

        for (i, (x, y)) in enumerate(trainLoader):
            pred = unet(x)
            loss = lossFunc(pred, y)
            
            train_epoch_loss.append(loss)

            opt.zero_grad()
            loss.backward()
            opt.step()

            
            pred_lst.append(pred.detach().numpy())
            target_lst.append(y.detach().numpy())
        
        pred_lst = np.concatenate(pred_lst, axis=0)
        target_lst =  np.concatenate(target_lst, axis=0)
        

        with torch.no_grad():
            unet.eval()
            for (x, y) in valLoader:
                pred = unet(x)
                pred_val_lst.append(pred.detach().numpy())
                val_epoch_loss.append(loss(pred,y))
            
        pred_val_lst = np.concatenate(pred_lst, axis=0)
                                    
        avgTrainLoss = train_epoch_loss.mean()
        avgTrainAccuracy = metrics.accuracy_score(target_lst, pred_lst.argmax(1))
        
        avgValLoss = val_epoch_loss.mean()
    
        train_accuracy.append(avgTrainAccuracy)
        val_accuracy.append(avgValAccuracy)
        train_loss.append(avgTrainLoss)
        val_loss.append(avgValLoss)
        
        
        print(f"Epoch {e}: Training Accuracy: {avgTrainAccuracy} Loss: {avgTrainLoss}")
        print(f"Epoch {e}: Validation Accuracy: {avgTrainAccuracy} Loss: {avgTrainLoss}")

    endTime = time.time()
    print("[INFO] total time taken to train the model: {:.2f}s".format(
        endTime - startTime))
    
    #Validation

In [ ]:
train_unet(u_net,trainLoader,valLoader,optimizer,lossfun,epoch_cnt = 100)